# Data Scientist Core Workflow

This notebook provides essential functionality for Data Scientists to explore and prepare drone imagery data for YOLOv11 model training. It focuses on core capabilities while maintaining simplicity.

## Workflow Overview

1. **Data Exploration**: Analyze and visualize the drone imagery dataset
2. **Data Preparation**: Prepare data for YOLOv11 training

## Prerequisites

- AWS account with appropriate permissions
- AWS CLI configured with "ab" profile
- SageMaker Studio access with Data Scientist role
- Access to the drone imagery dataset in S3 bucket: `lucaskle-ab3-project-pv`

Let's start by importing the necessary libraries and setting up our environment.

In [ ]:
import os
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import display, HTML
import io
from PIL import Image

# Set up AWS session with "ab" profile
session = boto3.Session(profile_name='ab')
s3_client = session.client('s3')
region = session.region_name
account_id = session.client('sts').get_caller_identity()['Account']

# Set up visualization
plt.rcParams["figure.figsize"] = (12, 6)

# Define bucket name
BUCKET_NAME = 'lucaskle-ab3-project-pv'

print(f"Data Bucket: {BUCKET_NAME}")
print(f"Region: {region}")
print(f"Account ID: {account_id}")

## 1. Data Exploration

Let's start by exploring the drone imagery dataset stored in S3.

In [ ]:
def list_s3_objects(bucket, prefix=""):
    """List all objects in an S3 bucket with the given prefix"""
    all_objects = []
    paginator = s3_client.get_paginator('list_objects_v2')
    
    # Create a PageIterator from the Paginator
    page_iterator = paginator.paginate(
        Bucket=bucket,
        Prefix=prefix
    )
    
    # Iterate through each page
    for page in page_iterator:
        if 'Contents' in page:
            all_objects.extend(page['Contents'])
    
    return all_objects

# Function to filter image files
def filter_image_files(objects):
    """Filter image files from S3 objects list"""
    image_extensions = [".jpg", ".jpeg", ".png", ".tiff", ".tif"]
    return [obj for obj in objects 
            if any(obj['Key'].lower().endswith(ext) for ext in image_extensions)]

# List raw images in the bucket
raw_objects = list_s3_objects(BUCKET_NAME, prefix="")
raw_images = filter_image_files(raw_objects)

print(f"Found {len(raw_images)} raw images in the bucket")

# Display the first few image keys
if raw_images:
    print("\nSample image keys:")
    for i, img in enumerate(raw_images[:5]):
        print(f"  {i+1}. {img['Key']}")

### 1.1 Display Sample Images

Let's display some sample images from the dataset to get a visual understanding.

In [ ]:
# Function to download and display images
def display_sample_images(bucket, image_objects, num_samples=4):
    """Download and display sample images from S3"""
    # Limit to the requested number of samples
    samples = image_objects[:min(num_samples, len(image_objects))]
    
    # Create a figure with subplots
    fig, axes = plt.subplots(1, len(samples), figsize=(16, 4))
    
    # If only one sample, axes is not an array
    if len(samples) == 1:
        axes = [axes]
    
    # Download and display each image
    for i, img_obj in enumerate(samples):
        try:
            # Download image from S3
            response = s3_client.get_object(Bucket=bucket, Key=img_obj['Key'])
            img_data = response['Body'].read()
            
            # Open image with PIL
            img = Image.open(io.BytesIO(img_data))
            
            # Display image
            axes[i].imshow(img)
            axes[i].set_title(os.path.basename(img_obj['Key']))
            axes[i].axis('off')
            
        except Exception as e:
            print(f"Error displaying image {img_obj['Key']}: {str(e)}")
            axes[i].text(0.5, 0.5, f"Error loading image", ha='center')
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Display sample images
if raw_images:
    display_sample_images(BUCKET_NAME, raw_images, num_samples=4)
else:
    print("No images found to display")

### 1.2 Basic Image Analysis

Let's analyze some basic characteristics of the images in our dataset.

In [ ]:
# Function to analyze image characteristics
def analyze_images(bucket, image_objects, sample_size=20):
    """Analyze basic characteristics of images"""
    # Limit to sample size
    samples = image_objects[:min(sample_size, len(image_objects))]
    
    # Initialize lists to store image characteristics
    widths = []
    heights = []
    aspect_ratios = []
    file_sizes = []
    formats = []
    
    print(f"Analyzing {len(samples)} sample images...")
    
    # Process each image
    for img_obj in samples:
        try:
            # Download image from S3
            response = s3_client.get_object(Bucket=bucket, Key=img_obj['Key'])
            img_data = response['Body'].read()
            
            # Get file size
            file_size = len(img_data) / (1024 * 1024)  # Convert to MB
            file_sizes.append(file_size)
            
            # Open image with PIL
            img = Image.open(io.BytesIO(img_data))
            
            # Get image dimensions
            width, height = img.size
            widths.append(width)
            heights.append(height)
            
            # Calculate aspect ratio
            aspect_ratio = width / height
            aspect_ratios.append(aspect_ratio)
            
            # Get image format
            formats.append(img.format)
            
        except Exception as e:
            print(f"Error analyzing image {img_obj['Key']}: {str(e)}")
    
    # Calculate statistics
    stats = {
        'count': len(widths),
        'avg_width': np.mean(widths) if widths else 0,
        'avg_height': np.mean(heights) if heights else 0,
        'min_width': min(widths) if widths else 0,
        'max_width': max(widths) if widths else 0,
        'min_height': min(heights) if heights else 0,
        'max_height': max(heights) if heights else 0,
        'avg_aspect_ratio': np.mean(aspect_ratios) if aspect_ratios else 0,
        'avg_file_size': np.mean(file_sizes) if file_sizes else 0,
        'formats': list(set(formats)) if formats else []
    }
    
    return {
        'stats': stats,
        'widths': widths,
        'heights': heights,
        'aspect_ratios': aspect_ratios,
        'file_sizes': file_sizes,
        'formats': formats
    }

# Analyze sample images
if raw_images:
    analysis_results = analyze_images(BUCKET_NAME, raw_images, sample_size=20)
    
    # Display statistics
    stats = analysis_results['stats']
    print("\nImage Statistics:")
    print(f"Total images analyzed: {stats['count']}")
    print(f"Average dimensions: {stats['avg_width']:.1f}x{stats['avg_height']:.1f} pixels")
    print(f"Dimension range: {stats['min_width']}x{stats['min_height']} to {stats['max_width']}x{stats['max_height']} pixels")
    print(f"Average aspect ratio: {stats['avg_aspect_ratio']:.2f}")
    print(f"Average file size: {stats['avg_file_size']:.2f} MB")
    print(f"Image formats: {', '.join(stats['formats'])}")
else:
    print("No images found to analyze")

### 1.3 Visualize Image Characteristics

Let's create some visualizations to better understand our dataset.

In [ ]:
# Visualize image characteristics
if 'analysis_results' in locals() and analysis_results['stats']['count'] > 0:
    # Create a figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot image dimensions
    axes[0, 0].scatter(analysis_results['widths'], analysis_results['heights'])
    axes[0, 0].set_xlabel('Width (pixels)')
    axes[0, 0].set_ylabel('Height (pixels)')
    axes[0, 0].set_title('Image Dimensions')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot aspect ratio distribution
    axes[0, 1].hist(analysis_results['aspect_ratios'], bins=10)
    axes[0, 1].set_xlabel('Aspect Ratio (width/height)')
    axes[0, 1].set_ylabel('Count')
    axes[0, 1].set_title('Aspect Ratio Distribution')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot file size distribution
    axes[1, 0].hist(analysis_results['file_sizes'], bins=10)
    axes[1, 0].set_xlabel('File Size (MB)')
    axes[1, 0].set_ylabel('Count')
    axes[1, 0].set_title('File Size Distribution')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot format distribution
    format_counts = {}
    for fmt in analysis_results['formats']:
        if fmt in format_counts:
            format_counts[fmt] += 1
        else:
            format_counts[fmt] = 1
    
    formats = list(format_counts.keys())
    counts = list(format_counts.values())
    
    axes[1, 1].bar(formats, counts)
    axes[1, 1].set_xlabel('Image Format')
    axes[1, 1].set_ylabel('Count')
    axes[1, 1].set_title('Image Format Distribution')
    
    plt.tight_layout()
    plt.show()
else:
    print("No analysis results available for visualization")

## 2. Data Preparation for YOLOv11 Training

Now let's prepare our data for YOLOv11 training. This involves organizing the data in the correct format and structure.

In [ ]:
# Function to check if labeled data exists
def check_labeled_data(bucket, prefix="labeled-data/"):
    """Check if labeled data exists in the bucket"""
    objects = list_s3_objects(bucket, prefix=prefix)
    
    if objects:
        print(f"Found {len(objects)} objects in labeled data directory")
        
        # Group by job name (assuming directory structure)
        jobs = {}
        for obj in objects:
            key = obj['Key']
            parts = key.split('/')
            if len(parts) > 2:
                job_name = parts[1]
                if job_name not in jobs:
                    jobs[job_name] = []
                jobs[job_name].append(key)
        
        # Display job information
        if jobs:
            print(f"\nFound {len(jobs)} labeling jobs:")
            for job, files in jobs.items():
                print(f"  - {job}: {len(files)} files")
        
        return jobs
    else:
        print("No labeled data found")
        return {}

# Check for labeled data
labeled_jobs = check_labeled_data(BUCKET_NAME)

### 2.1 Prepare Data Structure for YOLOv11

YOLOv11 requires a specific data structure. Let's prepare our data accordingly.

In [ ]:
# Function to create YOLO dataset structure
def prepare_yolo_structure(bucket, job_name=None):
    """Prepare YOLO dataset structure in S3"""
    # Define dataset structure
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    dataset_name = f"yolov11_dataset_{timestamp}"
    
    # Define directories
    base_prefix = f"datasets/{dataset_name}/"
    train_prefix = f"{base_prefix}train/"
    val_prefix = f"{base_prefix}val/"
    
    # Create empty directories in S3
    for prefix in [train_prefix, val_prefix]:
        for subdir in ["images/", "labels/"]:
            full_prefix = f"{prefix}{subdir}"
            # Create an empty object to represent the directory
            s3_client.put_object(Bucket=bucket, Key=full_prefix)
    
    print(f"Created YOLO dataset structure at s3://{bucket}/{base_prefix}")
    print("\nDirectory structure:")
    print(f"s3://{bucket}/{base_prefix}")
    print(f"├── train/")
    print(f"│   ├── images/")
    print(f"│   └── labels/")
    print(f"└── val/")
    print(f"    ├── images/")
    print(f"    └── labels/")
    
    return {
        'dataset_name': dataset_name,
        'base_prefix': base_prefix,
        'train_prefix': train_prefix,
        'val_prefix': val_prefix
    }

# Create YOLO dataset structure
yolo_structure = prepare_yolo_structure(BUCKET_NAME)

### 2.2 Data Preparation Instructions

To complete the data preparation for YOLOv11 training, follow these steps:

1. **Create Ground Truth Labeling Job**:
   - Use the `create_labeling_job.ipynb` notebook to create a labeling job
   - Label your images with bounding boxes for objects of interest

2. **Convert Ground Truth Output to YOLOv11 Format**:
   - After labeling is complete, convert the output to YOLOv11 format
   - Use the conversion function in the `create_labeling_job.ipynb` notebook

3. **Split Data into Train/Validation Sets**:
   - Split your labeled data into training and validation sets
   - Typically use 80% for training and 20% for validation

4. **Upload Data to YOLO Structure**:
   - Upload images to the `images/` directories
   - Upload corresponding label files to the `labels/` directories

5. **Create Dataset Configuration File**:
   - Create a YAML configuration file for your dataset
   - Specify paths to train and validation data
   - Define class names and IDs

Once these steps are complete, your data will be ready for YOLOv11 training.

## 3. Summary and Next Steps

In this notebook, we've explored the drone imagery dataset and transformed it for YOLOv11 training. Here's a summary of what we've accomplished:

1. **Data Exploration**:
   - Listed and displayed sample images from the S3 bucket
   - Analyzed image characteristics (dimensions, aspect ratios, file sizes)
   - Visualized image statistics

2. **Data Preparation**:
   - Checked for existing labeled data
   - Created YOLO dataset structure in S3

3. **Image Transformation** (NEW):
   - Discovered classes from raw-images/ directory structure
   - Split images into train/validation sets (80/20 split)
   - Created YOLO format labels for each image
   - Organized data in proper YOLOv11 structure
   - Generated data.yaml configuration file
   - Verified dataset integrity

### Key Features of the Transformation:

- **Automatic Class Discovery**: Discovers classes from S3 directory structure
- **Smart Label Generation**: Creates bounding boxes covering 80% of each image (centered)
- **Train/Val Split**: Randomly splits data with configurable ratio
- **YOLO Format**: Generates proper YOLO format labels with normalized coordinates
- **Data Verification**: Validates the created dataset structure
- **Configuration Files**: Creates data.yaml and dataset_info.json for reference

### Next Steps

1. **Use the transformed dataset** in the ML Engineer notebook for YOLOv11 training
2. **Fine-tune the bounding boxes** if needed (currently uses 80% centered boxes)
3. **Adjust train/val split ratio** if different proportions are needed
4. **Proceed to model training** using the generated dataset

### Dataset Ready for Training!

Your dataset is now in the proper YOLOv11 format and ready for training. The ML Engineer can use the dataset path provided in the transformation summary to train YOLOv11 models.

For more detailed functionality, refer to the comprehensive notebooks in the `notebooks/` directory.

### 2.2 Transform Classification Images to YOLOv11 Format

Let's transform the images from the classification structure (raw-images/class_name/) to YOLOv11 format with proper train/val splits and label files.

In [ ]:
import random
import yaml
import json
from collections import defaultdict

# Function to discover classes from raw-images structure
def discover_classes_from_s3(bucket, prefix="raw-images/"):
    """Discover class names from S3 directory structure"""
    response = s3_client.list_objects_v2(
        Bucket=bucket,
        Prefix=prefix,
        Delimiter='/'
    )
    
    classes = []
    if 'CommonPrefixes' in response:
        for obj in response['CommonPrefixes']:
            class_prefix = obj['Prefix']
            class_name = class_prefix.replace(prefix, '').rstrip('/')
            if class_name:  # Skip empty class names
                classes.append(class_name)
    
    return sorted(classes)

# Function to get images for each class
def get_images_by_class(bucket, prefix="raw-images/"):
    """Get all images organized by class"""
    classes = discover_classes_from_s3(bucket, prefix)
    images_by_class = {}
    
    print(f"Found {len(classes)} classes: {classes}")
    
    for class_name in classes:
        class_prefix = f"{prefix}{class_name}/"
        objects = list_s3_objects(bucket, prefix=class_prefix)
        images = filter_image_files(objects)
        images_by_class[class_name] = images
        print(f"  {class_name}: {len(images)} images")
    
    return images_by_class

# Discover classes and images
images_by_class = get_images_by_class(BUCKET_NAME)
class_names = list(images_by_class.keys())
total_images = sum(len(images) for images in images_by_class.values())

print(f"\nTotal images found: {total_images}")
print(f"Classes: {class_names}")

In [ ]:
# Function to create YOLO label file content
def create_yolo_label(class_id, image_width, image_height, 
                     bbox_x=None, bbox_y=None, bbox_w=None, bbox_h=None):
    """Create YOLO format label content
    
    For classification images, we'll create a bounding box that covers the entire image
    since we don't have specific object locations.
    
    Args:
        class_id: Class ID (0-based)
        image_width: Image width in pixels
        image_height: Image height in pixels
        bbox_x, bbox_y, bbox_w, bbox_h: Optional specific bounding box coordinates
    
    Returns:
        YOLO format label string
    """
    if bbox_x is None or bbox_y is None or bbox_w is None or bbox_h is None:
        # Create a bounding box covering most of the image (80% centered)
        # This assumes the object of interest is roughly centered in the image
        x_center = 0.5  # Center of image (normalized)
        y_center = 0.5  # Center of image (normalized)
        width = 0.8     # 80% of image width (normalized)
        height = 0.8    # 80% of image height (normalized)
    else:
        # Normalize the provided bounding box coordinates
        x_center = (bbox_x + bbox_w/2) / image_width
        y_center = (bbox_y + bbox_h/2) / image_height
        width = bbox_w / image_width
        height = bbox_h / image_height
    
    # Ensure values are within [0, 1] range
    x_center = max(0, min(1, x_center))
    y_center = max(0, min(1, y_center))
    width = max(0, min(1, width))
    height = max(0, min(1, height))
    
    return f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

# Function to split data into train/val sets
def split_train_val(images_by_class, train_ratio=0.8, random_seed=42):
    """Split images into train and validation sets"""
    random.seed(random_seed)
    
    train_data = {}
    val_data = {}
    
    for class_name, images in images_by_class.items():
        # Shuffle images
        shuffled_images = images.copy()
        random.shuffle(shuffled_images)
        
        # Split into train/val
        split_idx = int(len(shuffled_images) * train_ratio)
        train_data[class_name] = shuffled_images[:split_idx]
        val_data[class_name] = shuffled_images[split_idx:]
        
        print(f"{class_name}: {len(train_data[class_name])} train, {len(val_data[class_name])} val")
    
    return train_data, val_data

# Split data into train/val
if images_by_class:
    train_data, val_data = split_train_val(images_by_class)
    
    total_train = sum(len(images) for images in train_data.values())
    total_val = sum(len(images) for images in val_data.values())
    
    print(f"\nTotal train images: {total_train}")
    print(f"Total validation images: {total_val}")
else:
    print("No images found to split")

In [ ]:
# Function to transform and upload images to YOLO format
def transform_to_yolo_format(bucket, train_data, val_data, class_names, 
                           dataset_name, progress_callback=None):
    """Transform classification images to YOLO format and upload to S3"""
    
    # Create class ID mapping
    class_to_id = {class_name: idx for idx, class_name in enumerate(class_names)}
    
    # Define S3 prefixes
    base_prefix = f"datasets/{dataset_name}/"
    train_images_prefix = f"{base_prefix}train/images/"
    train_labels_prefix = f"{base_prefix}train/labels/"
    val_images_prefix = f"{base_prefix}val/images/"
    val_labels_prefix = f"{base_prefix}val/labels/"
    
    def process_split(data, images_prefix, labels_prefix, split_name):
        """Process a data split (train or val)"""
        processed_count = 0
        total_count = sum(len(images) for images in data.values())
        
        print(f"\nProcessing {split_name} split ({total_count} images)...")
        
        for class_name, images in data.items():
            class_id = class_to_id[class_name]
            
            for img_obj in images:
                try:
                    # Get original image key and filename
                    original_key = img_obj['Key']
                    filename = os.path.basename(original_key)
                    name_without_ext = os.path.splitext(filename)[0]
                    
                    # Download image to get dimensions
                    response = s3_client.get_object(Bucket=bucket, Key=original_key)
                    img_data = response['Body'].read()
                    img = Image.open(io.BytesIO(img_data))
                    width, height = img.size
                    
                    # Copy image to new location
                    new_image_key = f"{images_prefix}{filename}"
                    s3_client.copy_object(
                        Bucket=bucket,
                        CopySource={'Bucket': bucket, 'Key': original_key},
                        Key=new_image_key
                    )
                    
                    # Create YOLO label
                    label_content = create_yolo_label(class_id, width, height)
                    
                    # Upload label file
                    label_key = f"{labels_prefix}{name_without_ext}.txt"
                    s3_client.put_object(
                        Bucket=bucket,
                        Key=label_key,
                        Body=label_content.encode('utf-8')
                    )
                    
                    processed_count += 1
                    
                    # Progress callback
                    if progress_callback and processed_count % 10 == 0:
                        progress_callback(processed_count, total_count, split_name)
                        
                except Exception as e:
                    print(f"Error processing {original_key}: {str(e)}")
        
        print(f"Completed {split_name} split: {processed_count}/{total_count} images processed")
        return processed_count
    
    # Progress callback function
    def show_progress(current, total, split_name):
        percentage = (current / total) * 100
        print(f"  {split_name}: {current}/{total} ({percentage:.1f}%) processed")
    
    # Process train and validation splits
    train_processed = process_split(train_data, train_images_prefix, train_labels_prefix, "Train")
    val_processed = process_split(val_data, val_images_prefix, val_labels_prefix, "Validation")
    
    return {
        'train_processed': train_processed,
        'val_processed': val_processed,
        'class_to_id': class_to_id,
        'base_prefix': base_prefix
    }

# Transform images to YOLO format
if 'train_data' in locals() and 'val_data' in locals() and class_names:
    print("Starting transformation to YOLO format...")
    
    # Create a new dataset with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    dataset_name = f"yolov11_dataset_{timestamp}"
    
    transformation_result = transform_to_yolo_format(
        BUCKET_NAME, train_data, val_data, class_names, dataset_name
    )
    
    print(f"\nTransformation completed!")
    print(f"Dataset created at: s3://{BUCKET_NAME}/datasets/{dataset_name}/")
    print(f"Train images processed: {transformation_result['train_processed']}")
    print(f"Validation images processed: {transformation_result['val_processed']}")
else:
    print("No data available for transformation")

In [ ]:
# Function to create data.yaml configuration file
def create_data_yaml(bucket, dataset_name, class_names, class_to_id):
    """Create YOLO data.yaml configuration file"""
    
    # Create the YAML configuration
    data_config = {
        'path': f's3://{bucket}/datasets/{dataset_name}',
        'train': 'train/images',
        'val': 'val/images',
        'nc': len(class_names),
        'names': {class_to_id[name]: name for name in class_names}
    }
    
    # Convert to YAML string
    yaml_content = yaml.dump(data_config, default_flow_style=False, sort_keys=False)
    
    # Upload to S3
    yaml_key = f"datasets/{dataset_name}/data.yaml"
    s3_client.put_object(
        Bucket=bucket,
        Key=yaml_key,
        Body=yaml_content.encode('utf-8')
    )
    
    print(f"Created data.yaml at: s3://{bucket}/{yaml_key}")
    print("\nYAML content:")
    print(yaml_content)
    
    return yaml_key

# Create data.yaml file
if 'transformation_result' in locals():
    yaml_key = create_data_yaml(
        BUCKET_NAME, 
        dataset_name, 
        class_names, 
        transformation_result['class_to_id']
    )
else:
    print("No transformation result available to create data.yaml")

In [ ]:
# Function to verify the created dataset
def verify_yolo_dataset(bucket, dataset_name):
    """Verify the created YOLO dataset structure"""
    base_prefix = f"datasets/{dataset_name}/"
    
    # Check directory structure
    directories_to_check = [
        f"{base_prefix}train/images/",
        f"{base_prefix}train/labels/",
        f"{base_prefix}val/images/",
        f"{base_prefix}val/labels/"
    ]
    
    print(f"Verifying dataset: s3://{bucket}/{base_prefix}")
    print("=" * 60)
    
    total_files = 0
    
    for directory in directories_to_check:
        objects = list_s3_objects(bucket, prefix=directory)
        # Filter out directory markers (empty objects)
        files = [obj for obj in objects if obj['Size'] > 0]
        
        print(f"{directory}: {len(files)} files")
        total_files += len(files)
        
        # Show sample files
        if files:
            sample_files = files[:3]  # Show first 3 files
            for file_obj in sample_files:
                filename = os.path.basename(file_obj['Key'])
                size_kb = file_obj['Size'] / 1024
                print(f"  - {filename} ({size_kb:.1f} KB)")
            if len(files) > 3:
                print(f"  ... and {len(files) - 3} more files")
        print()
    
    # Check data.yaml
    yaml_key = f"{base_prefix}data.yaml"
    try:
        response = s3_client.get_object(Bucket=bucket, Key=yaml_key)
        yaml_content = response['Body'].read().decode('utf-8')
        print(f"data.yaml: Found ({len(yaml_content)} bytes)")
        print("Content preview:")
        print(yaml_content[:200] + "..." if len(yaml_content) > 200 else yaml_content)
    except Exception as e:
        print(f"data.yaml: Not found or error - {str(e)}")
    
    print(f"\nTotal files in dataset: {total_files}")
    
    # Verify image-label pairs
    train_images = list_s3_objects(bucket, prefix=f"{base_prefix}train/images/")
    train_labels = list_s3_objects(bucket, prefix=f"{base_prefix}train/labels/")
    val_images = list_s3_objects(bucket, prefix=f"{base_prefix}val/images/")
    val_labels = list_s3_objects(bucket, prefix=f"{base_prefix}val/labels/")
    
    # Filter out directory markers
    train_images = [obj for obj in train_images if obj['Size'] > 0]
    train_labels = [obj for obj in train_labels if obj['Size'] > 0]
    val_images = [obj for obj in val_images if obj['Size'] > 0]
    val_labels = [obj for obj in val_labels if obj['Size'] > 0]
    
    print(f"\nImage-Label Pair Verification:")
    print(f"Train: {len(train_images)} images, {len(train_labels)} labels")
    print(f"Val: {len(val_images)} images, {len(val_labels)} labels")
    
    # Check if counts match
    if len(train_images) == len(train_labels) and len(val_images) == len(val_labels):
        print("✅ Image-label pairs match!")
    else:
        print("⚠️  Image-label pair counts don't match!")
    
    return {
        'total_files': total_files,
        'train_images': len(train_images),
        'train_labels': len(train_labels),
        'val_images': len(val_images),
        'val_labels': len(val_labels)
    }

# Verify the created dataset
if 'dataset_name' in locals():
    verification_result = verify_yolo_dataset(BUCKET_NAME, dataset_name)
else:
    print("No dataset available to verify")

### 2.3 Dataset Summary and Usage Instructions

Your dataset has been successfully transformed to YOLOv11 format!

In [ ]:
# Display dataset summary
if 'dataset_name' in locals() and 'transformation_result' in locals():
    print("🎉 Dataset Transformation Complete!")
    print("=" * 50)
    print(f"Dataset Name: {dataset_name}")
    print(f"S3 Location: s3://{BUCKET_NAME}/datasets/{dataset_name}/")
    print(f"Classes: {len(class_names)} ({', '.join(class_names)})")
    print(f"Train Images: {transformation_result['train_processed']}")
    print(f"Validation Images: {transformation_result['val_processed']}")
    print(f"Total Images: {transformation_result['train_processed'] + transformation_result['val_processed']}")
    
    print("\n📁 Dataset Structure:")
    print(f"s3://{BUCKET_NAME}/datasets/{dataset_name}/")
    print("├── train/")
    print("│   ├── images/     # Training images")
    print("│   └── labels/     # Training labels (.txt files)")
    print("├── val/")
    print("│   ├── images/     # Validation images")
    print("│   └── labels/     # Validation labels (.txt files)")
    print("└── data.yaml       # Dataset configuration")
    
    print("\n🏷️  Class Mapping:")
    for class_name, class_id in transformation_result['class_to_id'].items():
        print(f"  {class_id}: {class_name}")
    
    print("\n📝 Label Format:")
    print("Each .txt file contains one line per object:")
    print("<class_id> <x_center> <y_center> <width> <height>")
    print("All coordinates are normalized (0.0 to 1.0)")
    
    print("\n🚀 Next Steps:")
    print("1. Use this dataset in the ML Engineer notebook for training")
    print("2. The dataset is ready for YOLOv11 training")
    print(f"3. Reference the dataset using: s3://{BUCKET_NAME}/datasets/{dataset_name}/data.yaml")
    
    # Save dataset info for later use
    dataset_info = {
        'dataset_name': dataset_name,
        'bucket': BUCKET_NAME,
        'base_path': f"s3://{BUCKET_NAME}/datasets/{dataset_name}/",
        'config_path': f"s3://{BUCKET_NAME}/datasets/{dataset_name}/data.yaml",
        'classes': class_names,
        'class_to_id': transformation_result['class_to_id'],
        'train_count': transformation_result['train_processed'],
        'val_count': transformation_result['val_processed'],
        'created_at': datetime.now().isoformat()
    }
    
    # Save dataset info to S3 for reference
    info_key = f"datasets/{dataset_name}/dataset_info.json"
    s3_client.put_object(
        Bucket=BUCKET_NAME,
        Key=info_key,
        Body=json.dumps(dataset_info, indent=2).encode('utf-8')
    )
    
    print(f"\n💾 Dataset info saved to: s3://{BUCKET_NAME}/{info_key}")
    
else:
    print("No dataset transformation was completed.")
    print("Please run the transformation cells above first.")